In [1]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
from data_clean import trimnoise

In [18]:
df = pd.read_csv('testWalk1.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.rename(columns={'Speed (m/s)': "speed"})
df = df.drop(df[df.Latitude == 0.0].index)
df_filtered = trimnoise(df)
df_filtered['time'] = df_filtered['time'].diff()
df_filtered['speed'] = df_filtered['speed'].diff()

In [19]:
groupSize = 32
grouped = df_filtered.groupby(df_filtered.index // groupSize)

In [23]:
df_grouped = grouped.mean()
#df_mean.drop(columns=['gFx', 'gFy', 'gFz'])
df_grouped = df_grouped[['time','speed','ax_f', 'ay_f', 'az_f','wx_f', 'wy_f', 'wz_f','gFx_f', 'gFy_f', 'gFz_f',]]

In [24]:
df_grouped['time'] = grouped.sum()['time']

In [189]:
rollingSize = 3

df_grouped['rolling_ax'] = df_grouped['ax_f'].rolling(rollingSize).sum()
df_grouped['rolling_ay'] = df_grouped['ay_f'].rolling(rollingSize).sum()
df_grouped['rolling_az'] = df_grouped['az_f'].rolling(rollingSize).sum()

In [191]:
df_grouped = df_grouped.dropna()

In [192]:
df_grouped.loc[df_grouped['speed'] > 0.0]

,time,speed,ax_f,ay_f,az_f,wx_f,wy_f,wz_f,gFx_f,gFy_f,gFz_f,rolling_ax,rolling_ay,rolling_az
10,0.412,0.038125,0.214933,-0.835303,0.450165,0.024097,-0.045051,0.112511,-0.058170,0.539145,0.885546,-0.512784,-1.753405,1.347595
13,0.355,0.008125,0.035306,-0.556406,0.454154,0.131091,-0.145837,-0.148950,-0.044675,0.408759,0.934362,-0.141917,-0.577879,-0.571204
15,0.352,0.004375,0.055244,0.021420,0.084085,0.395806,-0.144910,0.145122,-0.009748,0.569251,0.811008,-0.814290,-0.921071,1.275911
23,0.352,0.001563,0.601144,0.320297,-0.962388,0.147493,0.021614,0.349276,1.037055,-0.112992,0.109767,1.159535,-1.860533,0.282934
26,0.379,0.000937,0.843945,-0.782470,0.421919,-0.251150,0.211569,-0.174363,1.105255,-0.196431,0.243368,-2.204631,-1.475861,0.667509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118,0.393,0.000312,-0.154854,-0.929672,0.946816,0.153143,0.187077,-0.117217,0.931367,-0.291280,-0.187217,-0.677075,-0.269176,0.411633
2134,0.383,0.000312,0.790321,-1.000437,0.341652,-0.088185,-0.061051,0.047123,1.037252,-0.365815,-0.156594,0.069973,-0.071011,0.013022
2147,0.400,0.000312,0.553337,0.686382,-0.831605,-0.232458,0.041606,-0.105169,0.927096,-0.162574,-0.303720,0.214531,0.273783,0.091648
2162,0.400,0.000625,-0.509422,0.738972,-0.135907,-0.161012,-0.131820,-0.545522,0.139422,0.415785,0.869236,-0.345815,0.840916,1.867591


In [217]:
from sklearn.model_selection import train_test_split

X = df_grouped[['time', 'ax_f', 'ay_f','az_f', 'wx_f', 'wy_f','wz_f', 'gFx_f', 'gFy_f','gFz_f', 'rolling_ax', 'rolling_ay', 'rolling_az']].to_numpy()
y = df_grouped['speed'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [218]:
from sklearn.neural_network import MLPRegressor

mlp = make_pipeline( MLPRegressor(hidden_layer_sizes=(200,200,200,200,200),  activation='relu'))
result_mlp = mlp.fit(X_train, y_train)

In [219]:
result_mlp.score(X_test, y_test)

-1.4916553379349748

In [207]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
degree=2
polyreg=make_pipeline(PolynomialFeatures(degree),LinearRegression())

In [153]:
result = polyreg.fit(X_train, y_train)

In [154]:
result.score(X_test, y_test)

-0.056257258698512835

In [151]:
x_check = df_grouped.iloc[26][['time', 'ax_f', 'ay_f','az_f', 'wx_f', 'wy_f','wz_f']].to_numpy()

In [114]:
df_grouped.iloc[26]['speed']

0.0009374990625000031

In [115]:
result.predict([x_check])

array([0.0009375])

In [118]:
df2 = pd.read_csv('raw data/2020-12-06walka.csv')
df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')]
df2 = df2.rename(columns={'Speed (m/s)': "speed"})
df2 = df2.drop(df2[df2.Latitude == 0.0].index)
df2_filtered = trimnoise(df2)
df2_filtered['time'] = df2_filtered['time'].diff()
df2_filtered['speed'] = df2_filtered['speed'].diff()

grouped2 = df2_filtered.groupby(df2_filtered.index // groupSize)
df2_grouped = grouped2.mean()
#df_mean.drop(columns=['gFx', 'gFy', 'gFz'])
df2_grouped = df2_grouped[['time','speed','ax_f', 'ay_f', 'az_f','wx_f', 'wy_f', 'wz_f','gFx_f', 'gFy_f', 'gFz_f',]]
df2_grouped['time'] = df2_grouped.sum()['time']

In [119]:
df2_grouped

,time,speed,ax_f,ay_f,az_f,wx_f,wy_f,wz_f,gFx_f,gFy_f,gFz_f
0,13.101979,0.000000,0.079874,0.058910,-0.406934,0.098820,0.265989,0.481091,-0.080922,0.356911,0.913408
1,13.101979,0.000000,0.838844,0.186948,0.153593,0.425114,-0.153517,-0.577360,0.023061,0.457428,0.905683
2,13.101979,0.000000,0.006586,-0.817158,0.794498,0.086272,-1.162242,-1.237346,-0.034664,0.406010,0.913633
3,13.101979,0.000000,-2.234231,-0.108412,-0.566022,0.256615,-1.570037,2.606278,0.254976,0.304709,0.709620
4,13.101979,0.000000,1.361007,1.229393,-0.737402,0.472681,-1.409788,0.787184,0.874489,-0.147924,0.460118
...,...,...,...,...,...,...,...,...,...,...,...
1015,13.101979,-0.000312,3.587364,0.807406,-0.854563,-0.307425,0.092695,-1.667027,1.285028,-0.216329,0.070211
1016,13.101979,0.000000,-0.649649,-3.122801,0.092417,-1.327865,2.863021,-1.622465,0.255864,0.154456,0.769076
1017,13.101979,0.000000,0.330963,-0.264978,0.634920,-0.104974,-0.094380,0.050891,0.137148,0.326507,0.932004
1018,13.101979,0.000000,-0.102506,-0.126016,0.558346,-0.474515,0.177696,0.369797,0.118886,0.131680,1.022541


In [120]:
df2_grouped.loc[df2_grouped['speed'] > 0.0]

,time,speed,ax_f,ay_f,az_f,wx_f,wy_f,wz_f,gFx_f,gFy_f,gFz_f
54,13.101979,0.034375,-0.029042,-0.903905,0.600120,-0.087888,0.075530,0.005171,0.172906,-1.050518,-0.137946
56,13.101979,0.006250,0.356753,0.922444,0.783472,0.210588,0.330351,-0.020887,0.156826,-0.853866,-0.115504
61,13.101979,0.014687,-1.127668,0.868969,-0.242473,-0.300816,-0.220424,0.203178,-0.077527,-0.902343,-0.229335
64,13.101979,0.002500,-0.473000,1.059832,0.000550,-0.091526,-0.155911,0.152783,0.105740,-0.822508,-0.332103
72,13.101979,0.000312,0.420431,1.056960,-1.254167,0.047086,0.169781,0.118486,0.176648,-0.894525,-0.308842
...,...,...,...,...,...,...,...,...,...,...,...
935,13.101979,0.000312,0.201281,0.867093,-0.562537,-0.466406,-0.405651,0.221720,-0.071160,-0.878030,-0.391971
937,13.101979,0.000312,-0.036769,-1.034880,0.082818,-0.174339,0.055496,-0.019507,-0.044343,-1.118111,-0.143554
940,13.101979,0.000312,-0.340680,-1.337920,0.020074,-0.099535,0.143749,0.066250,-0.066718,-1.071337,-0.161912
945,13.101979,0.000312,-0.284608,0.754111,0.467701,0.726174,-0.135041,-0.399255,-0.054327,-0.846904,-0.168450


In [121]:
x2_check = df2_grouped.iloc[56][['time', 'ax_f', 'ay_f','az_f', 'wx_f', 'wy_f','wz_f']].to_numpy()
result.predict([x2_check])

array([-7.31147497e+08])

In [123]:
df2_grouped.iloc[56]['speed']

0.006249997812499998